In [22]:
from __future__ import print_function
import urllib
import bz2
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import torch

In [23]:
device = 'cpu'
device

'cpu'

In [24]:
##UNCOMMENT THE DATASET YOU WANT TO WORK WITH

BASE_URL = "http://grail.cs.washington.edu/projects/bal/data/ladybug/"
FILE_NAME = "problem-49-7776-pre.txt.bz2"

# BASE_URL = "http://grail.cs.washington.edu/projects/bal/data/ladybug/"
# FILE_NAME = "problem-73-11032-pre.txt.bz2"

# BASE_URL = "http://grail.cs.washington.edu/projects/bal/data/ladybug/"
# FILE_NAME = "problem-138-19878-pre.txt.bz2"


# BASE_URL = "http://grail.cs.washington.edu/projects/bal/data/trafalgar/"
# FILE_NAME = "problem-21-11315-pre.txt.bz2"

# BASE_URL = "http://grail.cs.washington.edu/projects/bal/data/trafalgar/"
# FILE_NAME = "problem-39-18060-pre.txt.bz2"

# BASE_URL = "http://grail.cs.washington.edu/projects/bal/data/trafalgar/"
# FILE_NAME = "problem-50-20431-pre.txt.bz2"



# BASE_URL = "http://grail.cs.washington.edu/projects/bal/data/dubrovnik/"
# FILE_NAME = "problem-16-22106-pre.txt.bz2"

# BASE_URL = "http://grail.cs.washington.edu/projects/bal/data/venice/"
# FILE_NAME = "problem-52-64053-pre.txt.bz2"

# BASE_URL = "http://grail.cs.washington.edu/projects/bal/data/final/"
# FILE_NAME = "problem-93-61203-pre.txt.bz2"



URL = BASE_URL + FILE_NAME

In [25]:
# Code to read in and process bundle adjustment data taken from https://scipy-cookbook.readthedocs.io/items/bundle_adjustment.html

In [26]:
'''Now read data from the file'''
def read_bal_data(file_name):
    with bz2.open(file_name, 'rt') as file:
        n_cameras, n_points, n_observations = map(
            int, file.readline().split()
        )

        camera_indices = torch.empty(n_observations, dtype=int)
        point_indices = torch.empty(n_observations, dtype=int)
        points_2d = torch.empty((n_observations, 2))

        for i in range(n_observations):
            camera_index, point_index, x, y = file.readline().split()
            camera_indices[i] = int(camera_index)
            point_indices[i] = int(point_index)
            points_2d[i] = torch.Tensor([float(x), float(y)])

        camera_params = torch.empty(n_cameras * 9)
        for i in range(n_cameras *9):
            camera_params[i] = float(file.readline())
        camera_params = torch.Tensor(camera_params.reshape((n_cameras, -1)))

        points_3d = torch.empty(n_points * 3)
        for i in range(n_points * 3):
            points_3d[i] = float(file.readline())
        points_3d = torch.Tensor(points_3d.reshape((n_points, -1)))

    return camera_params, points_3d, camera_indices, point_indices, points_2d


In [27]:
def rotate( points, rot_vecs ):
    """Rotation points by given rotation vectors
    Rodrigues's rotation formula is used
    """
    
    theta = torch.norm(rot_vecs,dim=1).reshape(-1,1)
    v = rot_vecs / theta
    
    dot = torch.sum(points * v,1).reshape(-1,1)
    cos_theta = torch.cos(theta)
    sin_theta = torch.sin(theta)

    return cos_theta * points + sin_theta * torch.cross(v, points) + dot * (1-cos_theta) * v

In [28]:
def project( points, camera_params):
    """Convert 3d points to 2d by projecting onto images"""
    points_proj = rotate(points, camera_params[:, :3]) 
    points_proj = points_proj+ camera_params[:, 3:6]
    points_proj = -points_proj[:, :2] / points_proj[:, 2].reshape(-1,1)
    f = camera_params[:, 6]
    k1 = camera_params[:, 7]
    k2 = camera_params[:, 8]
    n = torch.sum(torch.pow(points_proj,2), axis = 1)
    r = 1 + k1 * n +k2 * torch.square(n)
    points_proj = points_proj*(r*f).reshape(-1,1)
    return points_proj

In [29]:
def fun(params, n_cameras, n_points, camera_indices, point_indics, points_2d):
    camera_params = params[:n_cameras * 9].reshape((n_cameras, 9))
    points_3d = params[n_cameras * 9:].reshape((n_points, 3))
    points_proj = project(points_3d[point_indics], camera_params[camera_indices])
    return (points_proj - points_2d.to(device)).view(-1)

In [30]:
if not os.path.isfile(FILE_NAME):
    urllib.request.urlretrieve(URL, FILE_NAME)

camera_params, points_3d, camera_indices, point_indices, points_2d = read_bal_data(FILE_NAME)

camera_params = camera_params.to(device)
points_3d = points_3d.to(device)

camera_params.requires_grad = True
points_3d.requires_grad = True

n_cameras = camera_params.shape[0]
n_points = points_3d.shape[0]
n = 9 * n_cameras + 3 * n_points #num columns in jacobian 
m = 2 * points_2d.shape[0] #num rows in jacobian 

In [31]:
##create sparse matrix jacobian
def Jacobian(m,n):

    e[0].backward(retain_graph=True)

    row = torch.LongTensor(torch.empty(m*12).long())
    col = torch.LongTensor(torch.empty(m*12).long())

    jac_values = torch.empty(m*12)

    for i in range(len(e)):
        points_3d.grad.zero_()
        camera_params.grad.zero_()
        e[i].backward(retain_graph=True)
        
        with torch.no_grad():
            point_3d_index = int((points_3d.grad.nonzero())[0][0]) #which 3d point is involved
            camera_index = int((camera_params.grad.nonzero())[0][0])#which camera was involved

            row[i*12:i*12+12] = i
            col[i*12:i*12+12] = torch.cat((torch.linspace(n_cameras*9+point_3d_index, n_cameras*9+point_3d_index+2, steps=3),torch.linspace(camera_index*9, camera_index*9+8, steps=9)),0)
            
            jac_values[i*12: i*12+3] = points_3d.grad[point_3d_index] 

   
            jac_values[i*12+3:i*12+12] = camera_params.grad[camera_index] 

            
        if(i%5000==0): #print progress
            print("row: ",i," time elapsed: ",time.perf_counter()-start_time)
            
    return torch.sparse.FloatTensor(torch.cat((row,col),0).reshape(2,-1), jac_values, torch.Size([m,n])).to(device)   


In [32]:
## Implement LM
u=0.1 #beta multiply identity matrix
b = 100
V_prev = 0 #intialise performance index

for c in range(10):
    print("Iteration: ",c)
    start_time = time.perf_counter()
    x0 = torch.cat((torch.flatten(camera_params),torch.flatten(points_3d)),0)
    e = fun(x0, n_cameras, n_points, camera_indices, point_indices, points_2d).to(device)
    jacobian = Jacobian(m,n)
    print("Jacobian computed in: ",time.perf_counter()-start_time)

    with torch.no_grad():
        V_current = 0.5*torch.sum(torch.square(e.detach())) #performance index
        print("Performance Index: ",float(V_current),"b is",b)
        if(V_current > V_prev):
            u *= b
        else:
            u /= b
        V_prev = V_current
        torch.cuda.empty_cache()
        dParams = torch.matmul(torch.matmul(torch.inverse(torch.matmul(torch.transpose(jacobian.to_dense(),-1,0),jacobian.to_dense()) + u*torch.eye(n, n,device = device)),torch.transpose(jacobian.to_dense(),-1,0)),e)

        for i in range(len(camera_params)):
            camera_params[i] -= dParams[i*9:i*9+9]
        for i in range(len(points_3d)):
            points_3d[i] -= dParams[n_cameras*9+i*3:n_cameras*9+i*3+3]


Iteration:  0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)


row:  0  time elapsed:  0.34627190699995936


KeyboardInterrupt: ignored